In [1]:
import numpy as np
import pandas as pd
from numba import njit
import time
from pydivsufsort import divsufsort, kasai, common_substrings
import time
import numba

In [15]:
import threading
import time
import numpy as np

# Initialize a lock
lock = threading.Lock()

def safe_divsufsort_kasai(S):
    with lock:
        # Only one thread can enter this block at a time
        SA = get_SA(S)  # Assuming divsufsort is an external, optimized function
    return SA

@numba.njit
def process_first_loop(LCP, same_seq, n):
    f = np.zeros(n, dtype=np.int16)  # Initialize the result array f with zeros
    min_val = 0  # Initialize min to 0
    for i in range(1, n-2):
        if same_seq[i]:
            if LCP[i+1] < min_val:
                min_val = LCP[i+1]
            f[i + 1] = min_val
        else:
            min_val = LCP[i+1]
            f[i + 1] = LCP[i+1]
    return f

@numba.njit
def process_second_loop(LCP, same_seq, n):
    f = np.zeros(n, dtype=np.int16)  # Initialize the result array f with zeros
    min_val = 0  # Re-initialize min to 0 for the second loop
    for i in range(n-1, 1, -1):
        if same_seq[i-1]:  # Adjusted index for same_seq
            if LCP[i] < min_val:
                min_val = LCP[i]
            f[i - 1] = max(min_val, f[i - 1])
        else:
            min_val = LCP[i]
            f[i - 1] = max(min_val, f[i - 1])
    return f

def acs(A, B):
    start_time = time.time()

    S = f"{A}${B}"

    SA = safe_divsufsort_kasai(S)
    
    LCP = kasai(S, SA)
    LCP = np.append(-1, LCP[:-1])  # Assuming kasai is an external, optimized function
    
    print(f"SA: {SA}")
    print(f"LCP: {LCP}")
    
    n = len(S)
    mid = len(A) + 1

    # Compute same_seq array
    is_A = SA < mid
    is_A_shifted = np.roll(is_A, -1)
    same_seq = is_A & is_A_shifted
    same_seq |= (~is_A) & np.roll(~is_A, -1)    
    same_seq[-1] = False

    f1 = process_first_loop(LCP, same_seq, n)

    f2 = process_second_loop(LCP, same_seq, n)

    # Merge the results from both loops
    f = np.maximum(f1, f2)
    
    print(f"f: {f}")
    
    A_scores = f[is_A]
    B_scores = f[~is_A]
    
    d_AB = np.log(len(B))/np.mean(A_scores) - 2*np.log(len(A))/len(A)
    d_BA = np.log(len(A))/np.mean(B_scores) - 2*np.log(len(B))/len(B)
    
    d_ACS = (d_AB + d_BA)/2
    return d_ACS

In [16]:
acs("banana", "ananas")

SA: [ 6  5  3  1  7  9 11  0  4  2  8 10 12]
LCP: [-1  0  1  3  5  3  1  0  0  2  4  2  0]
f: [0 1 3 5 5 3 1 0 2 4 4 2 0]
[0 1 3 5 0 2 4]
[5 3 1 4 2 0]


0.17917594692280553

In [3]:
data = pd.read_parquet('../../data/processed/genomes.parquet', engine='pyarrow')  # You can use 'fastparquet' as the engine
data

,Accession ID,Lineage,Collection date,Sequence,Test
0,EPI_ISL_16823464,XBB.1.5,2023-01-31,TAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATC...,0
1,EPI_ISL_3342425,AY.116,2021-07-26,GTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGC...,0
2,EPI_ISL_1715410,B.1.525,2021-01-12,AGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCT...,1
3,EPI_ISL_515786,B.1.1.57,2020-07-29,TTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGT...,0
4,EPI_ISL_17385094,BQ.1.1,2023-02-06,TACGGCGCCGATCTAAAGTCATTTGACTTAGGCGACGAGCTTGGCA...,1
...,...,...,...,...,...
47317,EPI_ISL_15963061,CP.5,2022-11-01,TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTC...,1
47322,EPI_ISL_15963067,BE.7,2022-11-05,TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTC...,1
47324,EPI_ISL_15963069,BE.7,2022-11-05,CTAAACGANCTTTAAAATCTGTGTGGCTGTCNCTCGGCTGCATNCT...,1
47344,EPI_ISL_18407436,BA.1,2022-07-26,TTGTAGATCTGTTCTCTAAACGAACNTGAAAATCTGTGTGGCTGTC...,1


In [4]:
data['Sequence'] = data['Sequence'].str.replace('[^ACTG]', '', regex=True)

In [12]:
import numba
from numba import int64
from numba.typed import List, Dict

@numba.njit
def to_int_keys_best(l):
    seen = Dict.empty(
        key_type=numba.types.unicode_type,
        value_type=numba.types.boolean
    )
    ls = List()
    for e in l:
        if e not in seen:
            ls.append(e)
            seen[e] = True
    # Sort the list
    ls = sorted(ls)
    index = Dict.empty(
        key_type=numba.types.unicode_type,
        value_type=int64,
    )
    for i, v in enumerate(ls):
        index[v] = i
    return [index[v] for v in l]

@numba.njit
def custom_zip_longest(a, b, fillvalue=-1):
    max_len = max(len(a), len(b))
    result = List()
    for i in range(max_len):
        ai = a[i] if i < len(a) else fillvalue
        bi = b[i] if i < len(b) else fillvalue
        result.append((ai, bi))
    return result

@numba.njit
def suffix_array_best(s):
    n = len(s)
    k = 1
    line = to_int_keys_best(s)
    while max(line) < n - 1:
        temp = List()
        for a, b in custom_zip_longest(line, line[k:], fillvalue=-1):
            temp.append(a * (n + 1) + b + 1)
        line = to_int_keys_best(temp)
        k <<= 1
    return line

In [13]:
start = time.time()
suffix_array_best(data["Sequence"][0])
end = time.time()

print(end-start)

/tmp/ipykernel_11311/2338062813.py:45: NumbaTypeSafetyWarning: unsafe cast from int64 to unicode_type. Precision may be lost.
  line = to_int_keys_best(temp)


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1mNo implementation of function Function(<built-in function contains>) found for signature:
 
 >>> contains(DictType[unicode_type,bool]<iv=None>, int64)
 
There are 24 candidate implementations:
[1m   - Of which 22 did not match due to:
   Overload of function 'contains': File: <numerous>: Line N/A.
     With argument(s): '(DictType[unicode_type,bool]<iv=None>, int64)':[0m
[1m    No match.[0m
[1m   - Of which 2 did not match due to:
   Overload in function 'impl_contains': File: numba/typed/dictobject.py: Line 844.
     With argument(s): '(DictType[unicode_type,bool]<iv=None>, int64)':[0m
[1m    Rejected as the implementation raised a specific error:
      NumbaNotImplementedError: Failed in nopython mode pipeline (step: native lowering)
    [1m[1mCannot cast int64 to unicode_type: i64 %"arg.k"[0m
    [0m[1mDuring: 

In [5]:
def random_undersample(data_df, max_samples_per_class=1, random_state=42):
        
    undersampled_data = []

    for class_value, group in data_df.groupby('Lineage'):
        if len(group) > max_samples_per_class:
            undersampled_group = group.sample(n=max_samples_per_class, random_state=random_state)
        else:
            undersampled_group = group
        undersampled_data.append(undersampled_group)

    undersampled_data_df = pd.concat(undersampled_data)
    undersampled_data_df = undersampled_data_df.sample(frac=1, random_state=random_state).reset_index(drop=True)
    
    return undersampled_data_df

In [6]:
comparison_sequences = random_undersample(data)[:10]["Sequence"]

In [7]:
# start = time.time()
# acs_distances = []
# for genome in data["Sequence"][:10000]:
#     scores = np.zeros(len(comparison_sequences))
#     for i in range(len(comparison_sequences)):
#         scores[i] = acs(genome, comparison_sequences[i])
#     acs_distances.append(scores)
# end = time.time()

# print(end-start)

In [8]:
import concurrent.futures
import time
import numpy as np

def calculate_acs_for_genome(genome):
    scores = np.zeros(len(comparison_sequences))
    for i, comp_seq in enumerate(comparison_sequences):
        scores[i] = acs(genome, comp_seq)
    return scores

start = time.time()

# Assuming `data["Sequence"]` contains your genomes and is a Pandas Series or similar iterable
# and `comparison_sequences` is a list of sequences you are comparing against.
acs_distances = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit all genomes for processing, where each genome will be processed in parallel
    futures = [executor.submit(calculate_acs_for_genome, genome) for genome in data["Sequence"]]

    # As each future completes, collect the results
    for future in concurrent.futures.as_completed(futures):
        acs_distances.append(future.result())

end = time.time()

print(f"Time taken: {end - start} seconds")

/opt/conda/lib/python3.10/site-packages/pydivsufsort/divsufsort.py:75: UserWarning: converting str argument uses more memory
  warnings.warn("converting str argument uses more memory")
/tmp/ipykernel_9998/2052217322.py:53: UserWarning: converting str argument uses more memory
  LCP = kasai(S, SA)
/tmp/ipykernel_9998/2052217322.py:67: UserWarning: converting str argument uses more memory
  LCP = kasai(S, SA)
/opt/conda/lib/python3.10/site-packages/pydivsufsort/divsufsort.py:75: UserWarning: converting str argument uses more memory
  warnings.warn("converting str argument uses more memory")
/tmp/ipykernel_9998/2052217322.py:53: UserWarning: converting str argument uses more memory
  LCP = kasai(S, SA)
/tmp/ipykernel_9998/2052217322.py:67: UserWarning: converting str argument uses more memory
  LCP = kasai(S, SA)


Time taken: 417.01712250709534 seconds


In [9]:
acs_data = pd.DataFrame(acs_distances)
acs_data["Target"] = data["Lineage"].tolist()
acs_data["Test"] = data["Test"].tolist()
acs_data.to_parquet('../../data/features/acs.parquet', engine='pyarrow')

/opt/conda/lib/python3.10/site-packages/pandas/io/parquet.py:190: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)


In [10]:
# import pandas as pd
# import dask.dataframe as dd
# from dask.distributed import Client
# from dask import compute

# client = Client()

# # Define a wrapper function to apply 'ACS' over DataFrame rows
# def compare_genomes_to_all_comparisons(row, comparisons):
#     comparison_scores = []
#     for comp_seq in comparisons:
#         score = acs(row['Sequence'], comp_seq)
#         comparison_scores.append(score)
#     return comparison_scores

# # Convert your pandas DataFrame to a Dask DataFrame
# dask_df = dd.from_pandas(data, npartitions=64)  # You can adjust npartitions based on your system's cores

# # Use map_partitions to apply the function across all partitions in parallel
# # Note: We're using a lambda to pass the 'comparison_sequences' variable as a constant argument
# meta = pd.Series([], dtype=float)
# results = dask_df.map_partitions(lambda df: df.apply(compare_genomes_to_all_comparisons, axis=1, args=(comparison_sequences,)), meta=meta)

# start = time.time()
# # Compute the result
# computed_results = results.compute(scheduler='processes')
# end = time.time()

# # Convert the computed result to a pandas DataFrame
# final_df = computed_results.to_frame()

# print(final_df)
# print(end - start)

# expanded_df = pd.DataFrame(computed_results.apply(pd.Series))

# expanded_df["Target"] = data["Lineage"].tolist()
# expanded_df["Test"] = data["Test"].tolist()

# expanded_df.to_parquet('../../data/features/acs.parquet', engine='pyarrow')